# A: Data Collection

## Imports

In [143]:
# Install Libraries
# !pip install textblob
# !pip install tweepy
# !pip install pycountry
# !pip3 install nltk 
# !pip3 install -U setuptools wheel
# !pip3 install spacy 
# !pip3 install pandas numpy seaborn matplotlib
# !python3 -m spacy download en_core_web_sm
# !pip3 install gensim
# !pip install wordcloud
# !pip install langdetect
# !pip install pyquery
# !pip install lxml
# nltk.download('vader_lexicon')  
# !pip install requests
# !pip install schedule

In [20]:
# Import Libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
import requests
import json
import schedule
import time
import datetime
import csv
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer


### Authentication for Twitter API

In [17]:
# Authentication
consumer_Key = 'chie8F9xbOWpMDOxJTolC89yy'
consumer_Secret = 'WUQwRqXd7niAkOEBzZ5iOH76c4NCS6UunTNJ4IUSFuWempLcBQ'
access_Token = '1356292685385105408-yP2gaDQkkHMSSa2hUcjfwSKueKen8v'
access_Token_Secret = 'UlBapYsreywE4CyYARnEJmqDXlV5KrhXhgvWMmQGIoq3E'

auth = tweepy.OAuthHandler(consumer_Key, consumer_Secret)
auth.set_access_token(access_Token, access_Token_Secret)
api = tweepy.API(auth)

# curl -u 'chie8F9xbOWpMDOxJTolC89yy:WUQwRqXd7niAkOEBzZ5iOH76c4NCS6UunTNJ4IUSFuWempLcBQ' \
#   --data 'grant_type=client_credentials' \
#   'https://api.twitter.com/oauth2/token'

In [21]:
#its bad practice to place your bearer token directly into the script (this is just done for illustration purposes)
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAPgDNwEAAAAA5LZB%2F%2FVS6CkUQKsWmVwBVyQKXHk%3DUSoT6IfI2Fi052WlIqZjbOM1Duw1JTdm74ofZw6xvUwI1Z28sW"
#define search twitter function
def search_twitter(query, tweet_fields, expansions, place_fields, bearer_token = BEARER_TOKEN):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}

    url = "https://api.twitter.com/2/tweets/search/recent?query={}&{}&{}&{}".format(
        query, tweet_fields, expansions, place_fields
    )
    response = requests.request("GET", url, headers=headers)

    print(response.status_code)

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()
    print (response)

### Retrieving tweets according to hashtags

In [22]:
def isEnglish(text):
    try:
        text.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [23]:
def get_woeid(api, locations):
    twitter_world = api.trends_available()
    places = {loc['name'].lower() : loc['woeid'] for loc in twitter_world};
    woeids = []
    for location in locations:
        if location in places:
            woeids.append(places[location])
        else:
            print("err: ",location," woeid does not exist in trending topics")
    return woeids

In [11]:
get_woeid(api,'canada')

NameError: name 'api' is not defined

In [12]:
'''
Getting Tweets for the given hashtag with max of 1000 popular tweets with english dialect
'''
query = "#distancelearning"
def get_tweets(api, query):
    tweets = []
    for status in tweepy.Cursor(api.search,
                       q="#distancelearning",
                       count=10,
                       result_type='popular',
                       include_entities=True,
                       monitor_rate_limit=True, 
                       wait_on_rate_limit=True,
                       lang="en").items():
     
        # Getting only tweets which has english dialects
        if isEnglish(status.text) == True:
            tweets.append([status.id_str, query, status.created_at.strftime('%d-%m-%Y %H:%M'), status.user.screen_name, status.text])
    return tweets

In [ ]:
# def get_trending_hashtags(api, location):
#     woeids = get_woeid(api, location)
#     trending = set()
#     for woeid in woeids:
#         try:
#             trends = api.trends_place(woeid)
#         except:
#             print("API limit exceeded. Waiting for next hour")
#             #time.sleep(3605) # change to 5 for testing
#             trends = api.trends_place(woeid)
#         # Checking for English dialect Hashtags and storing text without #
#         topics = [trend['name'][1:] for trend in trends[0]['trends'] if (trend['name'].find('#') == 0 and isEnglish(trend['name']) == True)]
#         trending.update(topics)
    
#     return trending

In [24]:
def twitter_bot(api, locations):
    today = datetime.datetime.today().strftime("%d-%m-%Y-%s")
    if not os.path.exists("trending_tweets"):
        os.makedirs("trending_tweets")
    file_tweets = open("trending_tweets/"+today+"-tweets.csv", "a+")
    file_hashtags = open("trending_tweets/"+today+"-hashtags.csv", "w+")
    writer = csv.writer(file_tweets)
    
    hashtags = get_tweets(api, locations)
    file_hashtags.write("\n".join(hashtags))
    print("Hashtags written to file.")
    file_hashtags.close()
    
    for hashtag in hashtags:
        try:
            print("Getting Tweets for the hashtag: ", hashtag)
            tweets = get_tweets(api, "#"+hashtag)
        except:
            print("API limit exceeded. Waiting for next hour")
            time.sleep(3605) # change to 0.2 sec for testing
            tweets = get_tweets(api, "#"+hashtag)
        for tweet in tweets:
            writer.writerow(tweet)
    
    file_tweets.close()

In [25]:
twitter_bot(api, 'dubai')

Hashtags written to file.


In [26]:
def main():
    ''' 
    Use location = [] list for getting trending tags from different countries. 
    I have limited number of request hence I am using only 1 location
    '''
    # Authentication
    consumer_Key = 'chie8F9xbOWpMDOxJTolC89yy'
    consumer_Secret = 'WUQwRqXd7niAkOEBzZ5iOH76c4NCS6UunTNJ4IUSFuWempLcBQ'
    access_Token = '1356292685385105408-yP2gaDQkkHMSSa2hUcjfwSKueKen8v'
    access_Token_Secret = 'UlBapYsreywE4CyYARnEJmqDXlV5KrhXhgvWMmQGIoq3E'

    auth = tweepy.OAuthHandler(consumer_Key, consumer_Secret)
    auth.set_access_token(access_Token, access_Token_Secret)
    api = tweepy.API(auth)
    #locations = ['new york', 'los angeles', 'philadelphia', 'barcelona', 'canada', 'united kingdom', 'india']        
    locations = ['united arab emirates']
    # api = api()
    
    # schedule.every().day.at("00:00").do(twitter_bot, api, locations)
    schedule.every(10).seconds.do(twitter_bot, api, locations)
    while True:
        schedule.run_pending()
        time.sleep(1)
        
if __name__ == "__main__":
    main()

Hashtags written to file.
Hashtags written to file.
Hashtags written to file.


KeyboardInterrupt: 

In [35]:
#its bad practice to place your bearer token directly into the script (this is just done for illustration purposes)
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAPgDNwEAAAAA5LZB%2F%2FVS6CkUQKsWmVwBVyQKXHk%3DUSoT6IfI2Fi052WlIqZjbOM1Duw1JTdm74ofZw6xvUwI1Z28sW"
#define search twitter function
# place_fields = place_fields, 
def search_twitter(query, tweet_fields, place_fields, bearer_token = BEARER_TOKEN):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}

    # url = "https://api.twitter.com/2/tweets/search/recent?query={}&{}&{}".format(
    #     query, tweet_fields, place_fields 
    # )
    url = "https://api.twitter.com/1.1/geo/search/recent?query={}&{}&{}".format(query, tweet_fields, place_fields) 
    response = requests.request("GET", url, headers=headers)

    print(response.status_code)

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()
    print (response)

In [37]:
#its bad practice to place your bearer token directly into the script (this is just done for illustration purposes)
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAPgDNwEAAAAA5LZB%2F%2FVS6CkUQKsWmVwBVyQKXHk%3DUSoT6IfI2Fi052WlIqZjbOM1Duw1JTdm74ofZw6xvUwI1Z28sW"
#define search twitter function
# place_fields = place_fields, 
def search_twitter(query, tweet_fields, place_fields, bearer_token = BEARER_TOKEN):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}

    url = "https://api.twitter.com/2/tweets/search/recent?query={}&{}&{}".format(
        query, tweet_fields, place_fields 
    )
    # url = "https://api.twitter.com/1.1/geo/search/recent?query={}&{}&{}".format(query, tweet_fields, place_fields) 
    response = requests.request("GET", url, headers=headers)

    print(response.status_code)

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()
    print (response)

In [70]:
#search term
query = "(online school) OR (online learning) OR (distance learning) OR (online classes) OR (online class) OR (remote learning)"
#twitter fields to be returned by api call
tweet_fields = "tweet.fields=text,author_id,created_at,geo"
expansions = "expansions=geo.place_id"
place_fields = "place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type"

#twitter api call
json_response = search_twitter(query=query, tweet_fields=tweet_fields, place_fields=place_fields, bearer_token=BEARER_TOKEN)
# place_fields = place_fields, 
#pretty printing
# print(json.dumps(json_response['data'][0], indent=4, sort_keys=True))
# print(json_response.data[0]
headers = {"Authorization": "Bearer {}".format("AAAAAAAAAAAAAAAAAAAAAPgDNwEAAAAA5LZB%2F%2FVS6CkUQKsWmVwBVyQKXHk%3DUSoT6IfI2Fi052WlIqZjbOM1Duw1JTdm74ofZw6xvUwI1Z28sW")}
for i in range(0,len(json_response)):
    url = "https://api.twitter.com/1.1/geo/reverse_geocode.json?id={}&granularity=country".format(json_response['data'][i]['id'])
    response = requests.request("GET", url, headers=headers)
    print (response.json)

200
<bound method Response.json of <Response [200]>>
<bound method Response.json of <Response [200]>>


In [186]:
print(search_twitter.url)

AttributeError: 'function' object has no attribute 'url'

In [109]:
# Define the search term and the date_since date as variables
keyword = "#distancelearning" 
date_since = "2020-03-16"
noOfTweet = 100
geo = "25.204849,55.270782,100km"
new_search = keyword + " -filter:retweets" + "geocode:" + geo

In [112]:
#Sentiment Analysis
def percentage(part,whole):
 return 100 * float(part)/float(whole)
# keyword = input('Please enter keyword or hashtag to search: ')
# noOfTweet = int(input ('Please enter how many tweets to analyze: '))
tweets = tweepy.Cursor(api.search, q=new_search,lang="en", since=date_since).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

# users_locs = [[tweet.user.screen_name, tweet.user.location] for tweet in tweets]
# users_locs = []
for tweet in tweets:
    print(tweet.text)
    # print(tweet.user.location)
    # if (tweet.user.location=='Dubai'):
    #     users_locs.append(tweet.text)

    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
 
    if neg > pos:
        negative_list.append(tweet.text)
        negative += 1
    elif pos > neg:
        positive_list.append(tweet.text)
        positive += 1
 
    elif pos == neg:
        neutral_list.append(tweet.text)
        neutral += 1
positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

For teachers #distancelearning (or about to restart in-person) &amp; looking for ways to maintain or reawaken engagemen… https://t.co/aljLryUZ7B
RT @kat_goyette: Gain a fresh perspective/strategies for supporting educators toward effective technology integration: The Complete EdTech…
RT @kat_goyette: Looking for #distancelearning and #hybridlearning resources, based on protocol, foundation of #udl brain research? Lesson…
RT @WendellGMilesMD: Getting my meeting on with #aspsspringmeeting! #distancelearning is not just for the kids! #keeplearning #plasticsurge…
RT @danielbbudd: Machine Learning is very approachable within the Swift ecosystem - check out my series of tutorials to create an Image Cla…
RT @kat_goyette: Gain a fresh perspective/strategies for supporting educators toward effective technology integration: The Complete EdTech…
Gain a fresh perspective/strategies for supporting educators toward effective technology integration: The Complete… https://t.co/xuK8exfWX3
RT @danielbbu

In [111]:
print(tweet.text)

It’s important to seize the day, but always make time to go deeper and wider in your learning.

#studyinuk… https://t.co/qpkXkqJdkn
